This notebook is used to generate dataset for simulating stage one of BurstSketch

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from textwrap import wrap
from tqdm.notebook import tqdm
from multiprocessing import Pool

In [2]:
bursts = pd.read_csv('../BurstSketch/BurstDetection/bursts.csv')
data_flow = pd.read_csv('../BurstSketch/BurstDetection/input_flow.csv')

In [3]:
labels = []

burst_by_id = {int(fid): [] for fid in data_flow.id.unique()}
for row in bursts.itertuples():
    start, end, fid = row.start_window, row.end_window, row.flow_id
    
    for window in range(int(start), int(end) + 1):
        burst_by_id[fid].append(window)

for row in tqdm(data_flow.itertuples(), total=len(data_flow)):
    fid, window = row.id, row.window
    labels.append(window in burst_by_id[fid])

In [5]:
def string2list(string_id):
    return list(map(lambda x: int(x, 2), wrap('{0:064b}'.format(string_id), 8)))

string_ids = data_flow.id.values
pool_size = 12  # number of processes, see more at https://stackoverflow.com/questions/20039659/python-multiprocessings-pool-process-limit

with Pool(12) as p:
    formated_ids = p.map(string2list, string_ids)

In [8]:
dataset = pd.DataFrame(formated_ids)
dataset['label'] = labels

In [12]:
dataset.to_pickle('../data/stage_one_dataset.pickle')